In [61]:
import pandas as pd
import numpy as np
import ast

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import pairwise_distances

In [62]:
np.set_printoptions(threshold=1000)

In [63]:

movies = pd.read_csv("D:\\DataSet\\Recomedation System\\archive\\tmdb_5000_movies.csv")
credit = pd.read_csv("D:\\DataSet\\Recomedation System\\archive\\tmdb_5000_credits.csv")

In [64]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [65]:
credit.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [66]:
movies = movies[['genres','id','keywords','overview','title']]
credit = credit.drop(['movie_id'],axis=1)

In [67]:
movies.head(1)

,genres,id,keywords,overview,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar


In [68]:
movies = movies.merge(credit,how='inner',on='title')

In [69]:
movies.head(1)

,genres,id,keywords,overview,title,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [70]:
movies.isna().sum()

genres      0
id          0
keywords    0
overview    3
title       0
cast        0
crew        0
dtype: int64

In [71]:
movies.dropna(inplace=True)
credit.dropna(inplace=True)

In [72]:
movies.isna().sum()

genres      0
id          0
keywords    0
overview    0
title       0
cast        0
crew        0
dtype: int64

In [73]:
movies.head(1)

,genres,id,keywords,overview,title,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [74]:
movies.genres.iloc[2]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]'

In [75]:
movies.cast.iloc[0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [76]:
movies.crew.iloc[0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [77]:
def clean_genre(x):
    L = []
    for i in ast.literal_eval(x):
        L.append(i['name'])
    return L

In [78]:
def clean_cast(x):
    L = []
    counter = 0
    for i in ast.literal_eval(x):
        if counter < 3:
            counter += 1
            L.append(i['name'])
        else:
            break
    return L

In [79]:
def clean_crew(x):
    for i in ast.literal_eval(x):
        if i['job'] == 'Director':
            return [i['name']]

In [80]:
def remove_space(temp):
    l = []
    for i in temp:
        l.append(i.replace(' ',''))
    return l

In [81]:
movies['genres'] = movies['genres'].apply(clean_genre)

In [82]:
movies['keywords'] = movies['keywords'].apply(clean_genre)

In [83]:
movies['cast'] = movies['cast'].apply(clean_cast)

In [84]:
movies['crew'] = movies['crew'].apply(clean_crew)

In [85]:
movies.head(1)

,genres,id,keywords,overview,title,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [86]:
movies.overview.iloc[0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [87]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [88]:
movies.head(1)

,genres,id,keywords,overview,title,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [89]:
movies.dropna(inplace=True)

In [90]:
movies.isna().sum()

genres      0
id          0
keywords    0
overview    0
title       0
cast        0
crew        0
dtype: int64

In [91]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [92]:
movies['Tags'] = movies['overview'] + movies['keywords'] + movies['genres'] + movies['cast'] + movies['crew']

In [93]:
movies.head(1)

,genres,id,keywords,overview,title,cast,crew,Tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [ ]:
new_df = movies[['id','title','Tags']]

In [ ]:
new_df.head(1)

,id,title,Tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [ ]:
def stem(text):
    l = []
    for i in text:
        l.append(PorterStemmer().stem(i))
    return l

In [ ]:
new_df['Tags'] = new_df['Tags'].apply(stem)

C:\Users\Debjit\AppData\Local\Temp\ipykernel_23956\634297125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Tags'] = new_df['Tags'].apply(stem)


In [ ]:
new_df['Tags'] = new_df['Tags'].apply(lambda x: ' '.join(x))

C:\Users\Debjit\AppData\Local\Temp\ipykernel_23956\1615494922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Tags'] = new_df['Tags'].apply(lambda x: ' '.join(x))


In [ ]:
new_df.Tags.iloc[0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d action adventur fantasi sciencefict samworthington zoesaldana sigourneyweav jamescameron'

In [ ]:
cv = CountVectorizer(max_features = 5000,stop_words='english',lowercase=True)

vector = cv.fit_transform(new_df['Tags']).toarray()

In [ ]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      shape=(5000,), dtype=object)

In [ ]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4776, 5000))

In [ ]:
vector = pd.DataFrame(vector,index=movies['id'],columns=cv.get_feature_names_out())

In [ ]:
movies_sim = 1 - pairwise_distances(vector,metric='correlation',n_jobs=-1)

In [115]:
movies_sim_df = pd.DataFrame(movies_sim,index=new_df.index,columns=new_df.index)

In [116]:
movies_sim_df

,0,1,2,3,4,5,6,7,8,9,...,4799,4800,4801,4802,4803,4804,4805,4806,4807,4808
0,1.000000,0.077530,0.080653,0.066487,0.183662,0.102306,0.033765,0.140743,0.053901,0.090901,...,-0.004031,-0.004570,0.035732,0.046489,-0.006074,0.013960,0.043815,0.038227,-0.006184,-0.005923
1,0.077530,1.000000,0.054790,0.031164,0.068233,0.108195,0.014285,0.122798,0.057004,0.096134,...,-0.004263,-0.004833,0.015301,0.021154,-0.006423,0.035235,-0.003381,0.016466,-0.006540,0.020054
2,0.080653,0.054790,1.000000,0.053238,0.071184,0.064775,0.015581,0.127404,0.059450,0.099861,...,0.082231,-0.004393,-0.006533,-0.006233,0.011854,0.036990,-0.003073,0.017872,-0.005945,-0.005693
3,0.066487,0.031164,0.053238,1.000000,0.024721,0.052612,0.048320,0.060195,0.034650,0.196217,...,0.022653,0.021999,0.051077,0.047551,0.015213,0.061003,-0.003944,0.022935,0.041275,0.079678
4,0.183662,0.068233,0.071184,0.024721,1.000000,0.090447,0.047011,0.191488,0.073965,0.102176,...,0.030977,-0.005202,0.068813,0.016457,0.139241,0.150645,-0.003639,0.012324,0.076904,0.038051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,0.013960,0.035235,0.036990,0.061003,0.150645,0.073767,0.023669,0.137849,0.125656,0.029897,...,-0.003554,0.053774,0.133919,0.052376,0.255842,1.000000,-0.002818,-0.006248,0.147974,0.122131
4805,0.043815,-0.003381,-0.003073,-0.003944,-0.003639,-0.003565,-0.003525,-0.003673,-0.002962,-0.003371,...,-0.002105,0.067805,-0.003549,-0.003386,0.025663,-0.002818,1.000000,0.116987,-0.003229,-0.003093
4806,0.038227,0.016466,0.017872,0.022935,0.012324,0.010773,-0.007815,-0.008144,0.044412,0.013355,...,0.062966,0.028538,0.010328,0.037824,0.006865,-0.006248,0.116987,1.000000,0.032748,0.035733
4807,-0.006184,-0.006540,-0.005945,0.041275,0.076904,0.050770,0.028862,0.036276,0.099199,0.014914,...,-0.004072,0.030199,0.086768,-0.006550,0.136885,0.147974,-0.003229,0.032748,1.000000,0.081681


In [113]:
movies_sim_df[206647].sort_values(ascending=False)[0:5].index

Index([206647, 10764, 37724, 36670, 657], dtype='int64', name='id')

In [118]:
def get_similer_movies(movieid,nTop = 5):
    movieIdx = new_df[new_df.id == movieid].index
    if len(movieIdx) == 0:
        print('movie not found')
        return 
    else:
        movieIdx = movieIdx[0]
    new_df['similarity'] = movies_sim_df.iloc[movieIdx]
    ntop = new_df.sort_values(['similarity'],ascending=False)[0:nTop]
    return ntop['title'].values

In [121]:
get_similer_movies(285)

C:\Users\Debjit\AppData\Local\Temp\ipykernel_23956\1518056802.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['similarity'] = movies_sim_df.iloc[movieIdx]


array(["Pirates of the Caribbean: At World's End",
       "Pirates of the Caribbean: Dead Man's Chest",
       'Pirates of the Caribbean: The Curse of the Black Pearl',
       'Pirates of the Caribbean: On Stranger Tides', 'Life of Pi'],
      dtype=object)

In [131]:
name = pd.DataFrame(new_df.title.unique(),columns=['name'])

In [134]:
name.name.values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], shape=(4767,), dtype=object)